In [ ]:
from preprocess.preprocess import load_dataset, compute_label_agg, select_data, get_data_loaders, undersample

import torch
import torch.nn as nn
import torch.optim as optim

from models.CNN import *
from utils.trainer import trainer
from utils.RNN_utils import plot_losses, rename_with_acc

In [ ]:
# Define device for torch
use_cuda = True
device = torch.device("cuda" if (
    use_cuda and torch.cuda.is_available()) else "cpu")
print(device)

# Define options


In [ ]:
EPOCH = 20
LOAD_DATASET = False
TRAIN_PATH = './PTB_train.pt'
VALID_PATH = './PTB_valid.pt'
TEST_PATH = './PTB_test.pt'
SAVE_DIR = './model_saves/'
BATCH_SIZE = 256

# Loading the main dataset


In [ ]:
if LOAD_DATASET:
    train_loader = torch.load(TRAIN_PATH)
    valid_loader = torch.load(VALID_PATH)
    test_loader = torch.load(TEST_PATH)
else:
    path = './data/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3/'

    data, raw_labels = load_dataset(path)

    labels = compute_label_agg(raw_labels, path)

    data, labels, y = select_data(data, labels)

    data, labels, y = undersample(data, labels, y)

    train_loader, valid_loader, test_loader = get_data_loaders(
        data, labels, y, BATCH_SIZE)

    torch.save(train_loader, TRAIN_PATH)
    torch.save(valid_loader, VALID_PATH)
    torch.save(test_loader, TEST_PATH)

# Loading the augmented data loaders

In [ ]:
# train_loader_rn = torch.load('./trainloader_augmented_rn.pt')
# valid_loader_rn = torch.load('./validloader_augmented_rn.pt')
# test_loader_rm = torch.load('./testloader_augmented_rm.pt')

train_loader_rn = torch.load('./trainloader_augmented_rn.pt')
valid_loader_rn = torch.load('./validloader_augmented_rn.pt')
test_loader_rn = torch.load('./testloader_augmented_rn.pt')

# Combining the undersampled dataset with the Random Masking augmented dataset

In [ ]:
combined_train_rn_ds = torch.utils.data.ConcatDataset([train_loader.dataset, train_loader_rn.dataset])
combined_valid_rn_ds = torch.utils.data.ConcatDataset([valid_loader.dataset, valid_loader_rn.dataset])
combined_test_rn_ds = torch.utils.data.ConcatDataset([test_loader.dataset, test_loader_rn.dataset])

combined_rn_train_loader = torch.utils.data.DataLoader(combined_train_rn_ds, batch_size=BATCH_SIZE, shuffle=True)
combined_rn_valid_loader = torch.utils.data.DataLoader(combined_valid_rn_ds, batch_size=BATCH_SIZE)
combined_rn_test_loader = torch.utils.data.DataLoader(combined_test_rn_ds, batch_size=BATCH_SIZE)

# Training the model


In [ ]:
SAVE_NAME =  f'CNN_undersampled_augmented_rn_epoch_{EPOCH}'

model = CNN().to(device)
lr = 0.0001

train_accs, valid_accs, test_acc = trainer(model, combined_rn_train_loader, combined_rn_test_loader, combined_rn_valid_loader, lr = lr, num_epochs = EPOCH, save_dir=SAVE_DIR, model_save_name=SAVE_NAME)

# model = train_RNN(EPOCH, train_loader, valid_loader, model,
#                   criterion, optimizer, device=device, save_dir=SAVE_DIR, model_save_name=SAVE_NAME)

# Evaluate and plot losses


In [ ]:
# test_accuracy = evaluate_RNN(model, test_loader)
plot_losses(save_dir=SAVE_DIR, metrics_save_name=SAVE_NAME)
rename_with_acc(SAVE_NAME, SAVE_DIR, test_acc)

# Training the model


In [ ]:
SAVE_NAME =  f'MMCNN_CAT_undersampled_augmented_rn_epoch_{EPOCH}'

model = MMCNN_CAT().to(device)
lr = 0.0001

train_accs, valid_accs, test_acc = trainer(model, combined_rn_train_loader, combined_rn_test_loader, combined_rn_valid_loader, lr = lr, num_epochs = EPOCH, save_dir=SAVE_DIR, model_save_name=SAVE_NAME)

# model = train_RNN(EPOCH, train_loader, valid_loader, model,
#                   criterion, optimizer, device=device, save_dir=SAVE_DIR, model_save_name=SAVE_NAME)

# Evaluate and plot losses


In [ ]:
# test_accuracy = evaluate_RNN(model, test_loader)
plot_losses(save_dir=SAVE_DIR, metrics_save_name=SAVE_NAME)
rename_with_acc(SAVE_NAME, SAVE_DIR, test_acc)

# Training the model


In [ ]:
SAVE_NAME =  f'MMCNN_ATT_undersampled_augmented_rn_epoch_{EPOCH}'

model = MMCNN_ATT().to(device)
lr = 0.0001

train_accs, valid_accs, test_acc = trainer(model, combined_rn_train_loader, combined_rn_test_loader, combined_rn_valid_loader, lr = lr, num_epochs = EPOCH, save_dir=SAVE_DIR, model_save_name=SAVE_NAME)

# model = train_RNN(EPOCH, train_loader, valid_loader, model,
#                   criterion, optimizer, device=device, save_dir=SAVE_DIR, model_save_name=SAVE_NAME)

# Evaluate and plot losses


In [ ]:
# test_accuracy = evaluate_RNN(model, test_loader)
plot_losses(save_dir=SAVE_DIR, metrics_save_name=SAVE_NAME)
rename_with_acc(SAVE_NAME, SAVE_DIR, test_acc)

# Training the model


In [ ]:
SAVE_NAME =  f'MMCNN_SUM_Dropout_batchnorn_undersampled_augmented_rn_epoch_{EPOCH}'

model = MMCNN_SUM().to(device)
lr = 0.0001

train_accs, valid_accs, test_acc = trainer(model, combined_rn_train_loader, combined_rn_test_loader, combined_rn_valid_loader, lr = lr, num_epochs = EPOCH, save_dir=SAVE_DIR, model_save_name=SAVE_NAME)

# model = train_RNN(EPOCH, train_loader, valid_loader, model,
#                   criterion, optimizer, device=device, save_dir=SAVE_DIR, model_save_name=SAVE_NAME)

# Evaluate and plot losses


In [ ]:
# test_accuracy = evaluate_RNN(model, test_loader)
plot_losses(save_dir=SAVE_DIR, metrics_save_name=SAVE_NAME)
rename_with_acc(SAVE_NAME, SAVE_DIR, test_acc)

# Training the model


In [ ]:
SAVE_NAME =  f'MMCNN_SUM_ATT_undersampled_augmented_rn_epoch_{EPOCH}'

model = MMCNN_SUM_ATT().to(device)
lr = 0.0001

train_accs, valid_accs, test_acc = trainer(model, combined_rn_train_loader, combined_rn_test_loader, combined_rn_valid_loader, lr = lr, num_epochs = EPOCH, save_dir=SAVE_DIR, model_save_name=SAVE_NAME)

# model = train_RNN(EPOCH, train_loader, valid_loader, model,
#                   criterion, optimizer, device=device, save_dir=SAVE_DIR, model_save_name=SAVE_NAME)

# Evaluate and plot losses


In [ ]:
# test_accuracy = evaluate_RNN(model, test_loader)
plot_losses(save_dir=SAVE_DIR, metrics_save_name=SAVE_NAME)
rename_with_acc(SAVE_NAME, SAVE_DIR, test_acc)